In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 4
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 1e6
length = 20

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
#n_select = 10
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e

    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1000000.00, score:-57.32
 Observe Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:13.2, cum_e:13.2, new_i:11.8, cum_i:11.8
 True State Time: 1 | S: 12563950.0, E: 37.0, A: 0.0, I: 11.8, D: 0.0, R: 1.3, Budget: 1000000.00, score:-138.59
 Observe Time: 1 | S: 12563949.0, E: 36.0, A: 0.0, I: 11.8, D: 0.0, R: 0.0
 Time: 1 | new_e:18.6, cum_e:31.8, new_i:16.3, cum_i:28.2
 True State Time: 2 | S: 12563919.0, E: 49.9, A: 0.0, I: 28.1, D: 0.0, R: 2.9, Budget: 1000000.00, score:-254.10
 Observe Time: 2 | S: 12563919.0, E: 50.0, A: 0.0, I: 28.1, D: 0.0, R: 0.0
 Time: 2 | new_e:26.2, cum_e:58.0, new_i:22.8, cum_i:51.0
 True State Time: 3 | S: 12563875.3, E: 68.5, A: 0.0, I: 50.9, D: 0.0, R: 5.2, Budget: 1000000.00, score:-418.56
 Observe Time: 3 | S: 12563877.0, E: 68.0, A: 0.0, I: 50.9, D: 0.0, R: 3.0
 Time: 3 | new_e:37.2, cum_e:95.2, new_i:32.1, cum_i:83.1
 True State Time: 4

 True State Time: 32 | S: 11752751.2, E: 53.0, A: 36.8, I: 147.3, D: 1.3, R: 811010.3, Budget: 585420.02, score:-3266.47
 Observe Time: 32 | S: 11753023.0, E: 0.0, A: 36.8, I: 147.3, D: 1.3, R: 811001.0
 Time: 32 | new_e:23.4, cum_e:735.4, new_i:25.2, cum_i:956.5
 True State Time: 33 | S: 11729314.4, E: 63.0, A: 36.4, I: 154.1, D: 1.3, R: 834430.7, Budget: 573469.58, score:-3387.70
 Observe Time: 33 | S: 11729624.0, E: 0.0, A: 36.4, I: 154.1, D: 1.3, R: 834420.0
 Time: 33 | new_e:27.7, cum_e:763.1, new_i:29.1, cum_i:985.6
 True State Time: 34 | S: 11701942.8, E: 70.6, A: 41.4, I: 163.9, D: 1.3, R: 861779.9, Budget: 561400.97, score:-3523.15
 Observe Time: 34 | S: 11702298.0, E: 0.0, A: 41.4, I: 163.9, D: 1.3, R: 861767.0
 Time: 34 | new_e:31.0, cum_e:794.1, new_i:32.7, cum_i:1018.3
 True State Time: 35 | S: 11674489.1, E: 76.8, A: 49.1, I: 176.1, D: 1.4, R: 889207.5, Budget: 549568.52, score:-3670.13
 Observe Time: 35 | S: 11674895.0, E: 0.0, A: 49.1, I: 176.1, D: 1.4, R: 889193.0
 Tim

 True State Time: 60 | S: 11079951.7, E: 1437.0, A: 1149.4, I: 3007.7, D: 4.1, R: 1478450.1, Budget: 269831.17, score:-29465.20
 Observe Time: 60 | S: 11089077.0, E: 0.0, A: 1149.4, I: 3007.7, D: 4.1, R: 1478055.0
 Time: 60 | new_e:596.6, cum_e:6737.4, new_i:674.8, cum_i:7284.1
 True State Time: 61 | S: 11062379.3, E: 1523.7, A: 1264.6, I: 3306.2, D: 4.5, R: 1495521.7, Budget: 259267.43, score:-32226.09
 Observe Time: 61 | S: 11072476.0, E: 0.0, A: 1264.6, I: 3306.2, D: 4.5, R: 1495086.0
 Time: 61 | new_e:631.5, cum_e:7369.0, new_i:719.8, cum_i:8003.9
 True State Time: 62 | S: 11045134.6, E: 1649.8, A: 1344.6, I: 3612.3, D: 4.9, R: 1512253.9, Budget: 249005.79, score:-35210.79
 Observe Time: 62 | S: 11056257.0, E: 0.0, A: 1344.6, I: 3612.3, D: 4.9, R: 1511774.0
 Time: 62 | new_e:682.7, cum_e:8051.6, new_i:775.9, cum_i:8779.8
 True State Time: 63 | S: 11028105.4, E: 1815.6, A: 1404.5, I: 3936.2, D: 5.3, R: 1528732.9, Budget: 239549.33, score:-38490.16
 Observe Time: 63 | S: 11040336.0, 

 True State Time: 89 | S: 10475011.3, E: 145537.1, A: 3399.7, I: 179194.7, D: 304.8, R: 1760552.4, Budget: 32623.97, score:-1427296.58
 Observe Time: 89 | S: 10914516.0, E: 0.0, A: 3399.7, I: 179194.7, D: 304.8, R: 1732760.0
 Time: 89 | new_e:56761.9, cum_e:322426.9, new_i:53798.4, cum_i:312887.9
569.1715590953827


In [6]:
(time.time() - tic) / ( T - start_time + 1)


6.696168590994442